In [ ]:
%load_ext autoreload
%autoreload 2

from trading import api

trades_unit = api.fetch_data()
trades_date = api.convert_to_time(trades_unit)
trades_unit = api.compute_stats(trades_unit)
trades_date = api.compute_stats(trades_date.reset_index(), all_stats=False).set_index(('Date', '.'))

# print(trades_unit.columns)
# print(trades_unit.head())
# print(trades_date.columns)
# print(trades_date.head())

In [ ]:
import plotly.graph_objects as go
trades = trades_unit.copy()
periods = ['Globally', 'Yearly', 'Monthly', 'Weekly']

fig = go.Figure(go.Bar(x=trades.index.to_list(), y=trades[('Gain', 'Globally')].to_list(), name='Trades'))

for col in trades['Balance'].columns.get_level_values(0):
    for period in periods:
        if period in col:
            fig.add_trace(go.Scatter(x=trades.index.to_list(), y=trades[('Balance', col)].to_list(), name='Balance '+col.replace('_', ' ')))

fig.show()

In [ ]:
import pandas as pd
import numpy as np
trades = trades_unit.copy().ffill()
PERIOD = ['Globally', 'Yearly', 'Monthly', 'Weekly']

stats = pd.DataFrame()
for metric, period in trades.columns:
    for p in ['Globally', 'Yearly', 'Monthly', 'Weekly']:
        if p in period and metric in ['Count', 'Balance', 'CountTP', 'RateTP', 'PayoffTP', 'CountBE', 'RateBE', 'PayoffBE', 'CountSL', 'RateSL', 'PayoffSL']:
            stats.loc[period, metric] = trades[(metric, period)].iloc[-1]
stats['WinRate'] = stats['RateTP']
stats['PayoffRatio'] = stats['PayoffTP'] / -stats['PayoffSL']
stats

In [ ]:
gain = [col for col in trades.columns if 'Gain' in col] 
balance = [col for col in trades.columns if 'Balance' in col] 
trades_date = pd.concat([
    pd.DataFrame({'Date': pd.date_range(start=trades['Date'].min(), end=trades['Date'].max())}).set_index('Date'),
    trades.set_index('Date').groupby('Date')[gain].sum(),
    trades.set_index('Date').groupby('Date')[balance].last(),
], axis=1)
trades_date[[col for col in trades.columns if 'Balance_' in col]].head(50)